In [1]:
import sys
import cmath
import math
import os

if sys.platform=="cygwin":
    from cyglibra_core import *
elif sys.platform=="linux" or sys.platform=="linux2":
    from liblibra_core import *

from libra_py import units
from libra_py import data_conv
import libra_py.workflows.librax.defaults as defaults
import libra_py.workflows.librax.main as main

import libra_py.workflows.librax.x_to_libra_qe as x2liQE

import matplotlib.pyplot as plt   # plots
#import numpy as np
#from matplotlib.mlab import griddata

%matplotlib inline 

plt.rc('axes', titlesize=24)      # fontsize of the axes title
plt.rc('axes', labelsize=20)      # fontsize of the x and y labels
plt.rc('legend', fontsize=20)     # legend fontsize
plt.rc('xtick', labelsize=16)    # fontsize of the tick labels
plt.rc('ytick', labelsize=16)    # fontsize of the tick labels

plt.rc('figure.subplot', left=0.2)
plt.rc('figure.subplot', right=0.95)
plt.rc('figure.subplot', bottom=0.13)
plt.rc('figure.subplot', top=0.88)

colors = {}

colors.update({"11": "#8b1a0e"})  # red       
colors.update({"12": "#FF4500"})  # orangered 
colors.update({"13": "#B22222"})  # firebrick 
colors.update({"14": "#DC143C"})  # crimson   

colors.update({"21": "#5e9c36"})  # green
colors.update({"22": "#006400"})  # darkgreen  
colors.update({"23": "#228B22"})  # forestgreen
colors.update({"24": "#808000"})  # olive      

colors.update({"31": "#8A2BE2"})  # blueviolet
colors.update({"32": "#00008B"})  # darkblue  

colors.update({"41": "#2F4F4F"})  # darkslategray

clrs_index = ["11", "21", "31", "41", "12", "22", "32", "13","23", "14", "24"]

/home/alexey/miniconda2/envs/libra/lib/python2.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: to-Python converter for std::vector<std::complex<double>, std::allocator<std::complex<double> > > already registered; second conversion method ignored.
  if __name__ == '__main__':
/home/alexey/miniconda2/envs/libra/lib/python2.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: to-Python converter for std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > > already registered; second conversion method ignored.
  if __name__ == '__main__':
/home/alexey/miniconda2/envs/libra/lib/python2.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: to-Python converter for boost::python::detail::container_element<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, unsigned long, boost::python::detail::final_vector_derived_policies<std::vector<std::vector<int, std::allocator<in

In [2]:
params = {}

defaults.set_defaults(params, "QE")

for it in params.keys():
    print it, params[it]

Setting default parameters for Libra- QE  calculations
print_aux_results 1
tsh_method 1
sh_pop_ex_file_prefix /mnt/c/cygwin/home/Alexey-user/Programming/Project_libra/notebooks/Example13_libraX/res/sh_pop_ex
do_rescaling 1
mu_file_prefix /mnt/c/cygwin/home/Alexey-user/Programming/Project_libra/notebooks/Example13_libraX/res/mu
mo_ham /mnt/c/cygwin/home/Alexey-user/Programming/Project_libra/notebooks/Example13_libraX/mo_ham/
traj_file_prefix /mnt/c/cygwin/home/Alexey-user/Programming/Project_libra/notebooks/Example13_libraX/res/md
ent_file 
hop_probs /mnt/c/cygwin/home/Alexey-user/Programming/Project_libra/notebooks/Example13_libraX/hop_probs/
sh_pop_file_prefix /mnt/c/cygwin/home/Alexey-user/Programming/Project_libra/notebooks/Example13_libraX/res/sh_pop
sd_ham /mnt/c/cygwin/home/Alexey-user/Programming/Project_libra/notebooks/Example13_libraX/sd_ham/
print_sd_ham 0
print_coherences 0
therm None
QM_fraction 1.0
res /mnt/c/cygwin/home/Alexey-user/Programming/Project_libra/notebooks/Exam

In [3]:
params["nproc"] = 1        # the number of processors
params["dt_nucl"]=20.0     # time step for nuclear dynamics  ex) 20 a.u. = 0.5 fsec
params["Nsnaps"]=5         # the number of MD rounds
params["Nsteps"]=2         # the number of MD steps per snap
params["Ncool"] = -1
params["nspin"] = 2
params["electronic_smearing"] = 0.01 # Electronic smearing used in Fermi population calculation
params["nconfig"] = 1
params["el_mts"] = 1
params["num_SH_traj"] = 1
params["scf_itr"] = 10     # Number of SCF steps in each fractional occupation update
params["max_iteration"] = 30 # Maximum number of fractional Fermi cycle
params["non-orth"] = 1     # = 1 when MOs are non-orthogonal, = 0 when calculated in orthogonal MO basis
params["print_S_mat"] = 0  # 1 if S-matrix printing required, 0 if not required
params["smat_inc"] = 0     # 1 Including overlap matrix (S), 0 when overlap matrix (S) not included in el propagation

params["MD_type"] = 1      # 1 NVT ensamble, 0 NVE ensamble

# Thermostat parameters
params["therm"] = Thermostat({"thermostat_type":"Nose-Hoover","nu_therm":0.001,"Temperature":300.0,"NHC_size":3})
params["Temperature"] = params["therm"].Temperature
params["sigma_pos"] = 0.01  #Displace atomic position randomly
params["Nstart"] = 0

params["excitations"] = [ excitation(0,1,0,1), excitation(0,1,1,1) ] 
params["excitations_init"] = [1]
params["HOMO"] = 0
params["min_shift"] = 0
params["max_shift"] = 1 
for i in range(0,len(params["excitations"])):
    params["qe_inp0%i" %i] = "x%i.scf.in" %i    # initial input file
    params["qe_inp%i" %i] = "x%i.scf_wrk.in" %i # working input file 
    params["qe_out%i" %i] = "x%i.scf.out" %i    # output file


for it in params.keys():
    print it, params[it]

print_aux_results 1
tsh_method 1
sh_pop_ex_file_prefix /mnt/c/cygwin/home/Alexey-user/Programming/Project_libra/notebooks/Example13_libraX/res/sh_pop_ex
nspin 2
do_rescaling 1
dt_nucl 20.0
mu_file_prefix /mnt/c/cygwin/home/Alexey-user/Programming/Project_libra/notebooks/Example13_libraX/res/mu
mo_ham /mnt/c/cygwin/home/Alexey-user/Programming/Project_libra/notebooks/Example13_libraX/mo_ham/
excitations_init [1]
traj_file_prefix /mnt/c/cygwin/home/Alexey-user/Programming/Project_libra/notebooks/Example13_libraX/res/md
ent_file 
hop_probs /mnt/c/cygwin/home/Alexey-user/Programming/Project_libra/notebooks/Example13_libraX/hop_probs/
sh_pop_file_prefix /mnt/c/cygwin/home/Alexey-user/Programming/Project_libra/notebooks/Example13_libraX/res/sh_pop
sd_ham /mnt/c/cygwin/home/Alexey-user/Programming/Project_libra/notebooks/Example13_libraX/sd_ham/
print_sd_ham 0
excitations [<liblibra_core.excitation object at 0x7f374cdc53c0>, <liblibra_core.excitation object at 0x7f374cdc52b8>]
print_coherence

In [6]:
########### Now start actual calculations ###########################

qe_run = { "EXE"       :"/mnt/c/cygwin/home/Alexey-user/Soft/espresso/bin/pw.x", 
           "EXE_EXPORT":"/mnt/c/cygwin/home/Alexey-user/Soft/espresso/bin/pw_export.x",
           "BATCH_SYSTEM":None,
           "NP": 1
         }


nstates = len(params["excitations"])
for ex_st in xrange(nstates):
    os.system("cp x%i.scf.in x%i.scf_wrk.in" % (ex_st, ex_st))
    x2liQE.exe_espresso(ex_st, qe_run)